<a href="https://colab.research.google.com/github/anirudh5698/Food-volume-estimation/blob/main/food_volume_github.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Required Installations:


In [ ]:
!pip install -U torch==1.5 torchvision==0.6 -f https://download.pytorch.org/whl/cu101/torch_stable.html 
!pip install cython pyyaml==5.1
!pip install -U 'git+https://github.com/cocodataset/cocoapi.git#subdirectory=PythonAPI'
import torch, torchvision
print(torch.__version__, torch.cuda.is_available())
!gcc --version   

# install detectron2:
!pip install detectron2==0.1.2 -f https://dl.fbaipublicfiles.com/detectron2/wheels/cu101/index.html  
!pip install imantics



Importing required Libraries:


In [ ]:
# You may need to restart your runtime prior to this, to let your installation take effect
# Some basic setup:
# Setup detectron2 logger
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

# import some common libraries
import numpy as np
import pandas as pd
import cv2
import json
from tqdm.notebook import tqdm

# import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog
from detectron2.utils.visualizer import ColorMode
from detectron2.data.datasets import register_coco_instances
from detectron2.engine import DefaultTrainer
from detectron2.evaluation import COCOEvaluator, inference_on_dataset
from detectron2.data import build_detection_test_loader

# For reading annotations file
from pycocotools.coco import COCO

# utilities
from pprint import pprint # For beautiful print!
import os 

# For data visualisation
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import plotly.express as px
from google.colab.patches import cv2_imshow
from detectron2.data.datasets import register_coco_instances

Path Address:


In [ ]:
training_images_path="/content/drive/MyDrive/dataset2/train"
testing_images_path="/content/drive/MyDrive/dataset2/val"
training_json="/content/drive/MyDrive/train.json"
testing_json="/content/drive/MyDrive/val.json"
eval_images_path="/content/drive/MyDrive/eval_data/Images"
eval_json="/content/drive/MyDrive/eval.json"

In [ ]:
train_coco=COCO(training_json)
category_ids = train_coco.loadCats(train_coco.getCatIds())

category_names = [_["name"] for _ in category_ids]

pprint(", ".join(category_names))


Registering Coco Instances


In [ ]:
register_coco_instances("training",{},training_json,training_images_path)
register_coco_instances("testing",{},testing_json,testing_images_path)
register_coco_instances("evaluation",{},eval_json,eval_images_path)

In [ ]:
from detectron2.engine import DefaultTrainer
from detectron2.config import get_cfg

cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"))
cfg.DATASETS.TRAIN = ("training",)
cfg.DATASETS.TEST = ()
cfg.DATALOADER.NUM_WORKERS = 2

#cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml")
cfg.MODEL.WEIGHTS="/content/drive/MyDrive/model_final(1).pth"
cfg.SOLVER.IMS_PER_BATCH = 2
cfg.SOLVER.BASE_LR = 0.00025
cfg.SOLVER.MAX_ITER = 20000
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 103

In [ ]:
microcontroller_metadata = MetadataCatalog.get("training")

In [ ]:
from detectron2.utils.visualizer import ColorMode

#Use the final weights generated after successful training for inference  
#cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")
cfg.MODEL.WEIGHTS="/content/drive/MyDrive/model_final (1).pth"
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.20  # set the testing threshold for this model
#Pass the validation dataset
cfg.DATASETS.TEST = ("testing", )

predictor = DefaultPredictor(cfg)
im = cv2.imread("/content/test.jpg")
outputs = predictor(im)
v = Visualizer(im[:, :, ::-1],
                   metadata=microcontroller_metadata, 
                   scale=0.8,
                   instance_mode=ColorMode.IMAGE   
    )
v = v.draw_instance_predictions(outputs["instances"].to("cpu")) #Passing the predictions to CPU from the GPU
cv2_imshow(v.get_image()[:, :, ::-1])



In [ ]:
def getList(dict):
    return dict.keys()
dict = outputs
print(getList(dict))
a=outputs["instances"]
b=a.get_fields()
scores=b['scores'].cpu().numpy()
rois=b['pred_boxes'].tensor.cpu().numpy()
pred_class=b['pred_classes'].cpu().numpy()
pred_masks=b['pred_masks'].cpu().numpy()
min_conf=0.30
import tensorflow as tf
from skimage import io
masking=pred_masks
image_list=[]
image_names=[]

masking=masking.astype(int)
print(masking.shape)
for i in range(masking.shape[0]):
  if scores[i] > min_conf and category_names[pred_class[i]]!="Background":
    temp= io.imread('/content/test.jpg')
    for j in range(temp.shape[2]):
        temp[:,:,j] = temp[:,:,j] * masking[i,:,:]
    plt.figure(figsize=(4,4))
    
    
    temp=cv2.cvtColor(temp, cv2.COLOR_BGR2GRAY)
    globals()[f"temp_{i}"] = temp
    image_list.append(i)
    b=cv2.countNonZero(temp)
    print("confidence: ",100*scores[i],"   ","Class: ",category_names[pred_class[i]])
    image_names.append(category_names[pred_class[i]])
    plt.imshow(temp)

import numpy as np
from imantics import Polygons, Mask
for i in image_list:
  temp=globals()[f"temp_{i}"]
  globals()[f"polygons_points_{i}"] = Mask(temp).polygons()
  globals()[f"polygons_cords_{i}"]=globals()[f"polygons_points_{i}"].points[0]




In [ ]:
plate_cords=[]
for i in image_list:
  globals()[f"polygons_cords_{i}"]=globals()[f"polygons_cords_{i}"].tolist()
plate ={
      "label":"plate",
      "line_color" : "null",
      "fill_color": "null",
      "points" : plate_cords,
     "shape_type" : "polygon"
    
    
}
shapes=[]
j=0
for i in image_list:
  
  a={
      "label":str(image_names[j]),
      "line_color" : "null",
      "fill_color": "null",
      "points" : globals()[f"polygons_cords_{i}"],
     "shape_type" : "polygon"
     
       
  }
  shapes.append(a)
  j=j+1
shapes.append(plate)
import json
data={
    "version": "3.5.0",
    "flags": {},
    "shapes" : shapes,
    "lineColor": [
    0,
    255,
    0,
    128
  ],
  "fillColor": [
    255,
    0,
    0,
    128
  ]

      }
json_object = json.dumps(data, indent = 4)
with open("data.json", "w") as outfile:
    outfile.write(json_object)




In [ ]:
cd "/content/drive/MyDrive/Food-Volume-Estimation/Volume Estimation"
!python demo.py --img "/content/test.jpg"  --json "/content/drive/MyDrive/Food-Volume-Estimation/Volume Estimation/data.json" --output "/content/drive/MyDrive/Food-Volume-Estimation/Volume Estimation/output"
